In [4]:
#libraries
import pandas as pd
import numpy as np
from prettytable import PrettyTable
import seaborn as sns
import matplotlib.pyplot as plt
import re
import regex
import string
from sklearn.feature_extraction.text import CountVectorizer

from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
#installing tweet-preprocessor
from tqdm.auto import tqdm
!pip install tweet-preprocessor --user
# import preprocessor as p
from pathlib import Path

#emoji library
# !pip install -e emoji
# import emoji

# model save
import joblib

#split
from sklearn.model_selection import train_test_split

! pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
     |████████████████████████████████| 133kB 2.7MB/s 


In [6]:
class featurise(object):
    def __init__(self, df):
        super(featurise, self).__init__()
        self.x = df['tweets']
        self.y = df['target']
        self.intensity = df['target_intensity']
        ##########################################
        self.final_df = pd.DataFrame()
        self.final_df['tweets'] = self.x
        self.final_df['target'] = self.y
        self.final_df['target_intensity'] = self.intensity
        ##########################################
        self.stop_words = stopwords.words('english')
        self.stop_words.remove('not')
        ##########################################
        # call process here
        # return self.process()

    def valder_scores(self, x):
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(x)
        return sentiment_dict['neg'], sentiment_dict['neu'], sentiment_dict['pos'], sentiment_dict['compound']

    def polar_word_count_mpqa(self, x):
        mpqa_df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/mpqa.txt",header=None, sep="\t")
        dic = mpqa_df.set_index(0).to_dict()
        pos,neg = 0,0
        for each in  word_tokenize(x):
            if each.lower() in dic[1].keys():
                if dic[1][each.lower()] == 'negative': neg+=1
                else: pos+=1
        return pos,neg
    
    def polar_word_count_bingliu(self, x):
        bingliu_df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/1. BingLiu.csv", header=None, sep="\t")
        dic = bingliu_df.set_index(0).to_dict()
        pos,neg = 0,0
        for each in  word_tokenize(x):
            if each.lower() in dic[1].keys():
                if dic[1][each.lower()] == 'negative': neg+=1
                else: pos+=1
        return pos,neg

    def agg_polarity_score_sent140(self,x):
        sent140_df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/3. Sentiment140-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        dic = sent140_df.set_index(0).to_dict()
        pos,neg = 0,0
        for each in  word_tokenize(x.lower()):
            if each in dic[2].keys():
                pos+=dic[2][each]
            if each in dic[3].keys():
                neg+=dic[3][each]
        return pos, neg
    
    def nrc_hashtag_score(self,x):
        df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/7. NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        dic = df.set_index(0).to_dict()
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in dic[1].keys():
                score+=dic[1][each]
        return score
    
    def nrc_emotion_score(self,x):
        df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/7. NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        dic = df.set_index(0).to_dict()
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in dic[1].keys():
                score+=dic[1][each]
        return score

    def nrc_emotion_score(self,x):
        df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/8. NRC-word-emotion-lexicon.txt", header=None, sep="\t")
        dic = df.set_index(0).to_dict()
        score = 0
        # do stuff prepare a df like bow and compute the freq of each emotion
        return score

    def affin_word_score(self,x):
        df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/AFINN-111.txt", header=None, sep="\t")
        dic = df.set_index(0).to_dict()
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in dic[1].keys():
                score+=dic[1][each]
        return score
    
    def agg_emotion_nrc10ext(self,x):
        df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/AFINN-111.txt", sep="\t")
        # df['sum'] = df['anger']+df['anticipation']+df['disgust']+df['fear']+df['joy']+df['negative']+df['positive']+df['sadness']+df['surprise']+df['trust']
        # df = df[["word","sum"]]
        # dic = df.set_index('word').to_dict()
        # score = 0
        for each in  word_tokenize(x.lower()):
            if each in dic['sum'].keys():
                score+=dic['sum'][each]
        return score

    def sentiwordnet(self,x):
        
        return None
        
    def agg_hash_nrc10ext(self,x):
        df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/5. NRC-Hashtag-Emotion-Lexicon-v0.2.txt", header=None, sep="\t")
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in set(df[1]):
                score+=sum(list(df[df[1]==str(each)][2]))
        return score
    
    def emoticon_afinn(self,x):
        afinn_df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/9. AFINN-emoticon-8.txt",header=None, sep="\t")
        dic = afinn_df.set_index(0).to_dict()
        score = 0
        for each in  word_tokenize(x):
            if each in df[1]:
                score+=dic[1][each]
        return score
    
    def process():
        return self


#unigrams bow
def bow(X_train, X_test, X_cv, y_train, y_test, y_cv):

    count_vect = CountVectorizer() #in scikit-learn
    count_vect.fit(X_train)

    bow_counts_train = count_vect.transform(X_train)
    bow_counts_cv = count_vect.transform(X_cv)
    bow_counts_test = count_vect.transform(X_test)
    print("the shape of out text BOW vectorizer ",bow_counts_train.get_shape())
    print("the number of unique words ", bow_counts_train.get_shape()[1])
    # final_counts_test=final_counts_test.toarray()
    return bow_counts_train, bow_counts_cv, bow_counts_test

#bigrams bow
def bigram(X_train, X_test, X_cv, y_train, y_test, y_cv):
    count_vect = CountVectorizer(ngram_range=(2,2))
    bigram_counts_train = count_vect.fit_transform(X_train)
    bigram_counts_cv = count_vect.transform(X_cv)
    bigram_counts_test = count_vect.transform(X_test)
    print("some feature names ", count_vect.get_feature_names()[:10])
    print("the shape of out  BIGRAM vectorizer ",bigram_counts_train.get_shape())
    print("the number of unique words with bigrams ", bigram_counts_train.get_shape()[1])
    return bigram_counts_train, bigram_counts_cv, bigram_counts_test


In [7]:
columns = ['id', "tweets", "target", "target_intensity"]
joy_train = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/a3_b_data/joy-ratings-0to1.train.txt", sep="\t", header=None, names=columns)
joy_test = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/a3_b_data/joy-ratings-0to1.test.gold.txt", sep="\t", header=None, names=columns)
anger_train = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/a3_b_data/anger-ratings-0to1.train.txt", sep="\t", header=None, names=columns)
anger_test = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/a3_b_data/anger-ratings-0to1.test.gold.txt", sep="\t", header=None, names=columns)


In [12]:
df = pd.read_csv("/content/drive/My Drive/IIITD Data/NLP/lexicons/4. SentiWordNet_3.0.0_20130122.txt", sep="\t")
# df['sum'] = df['anger']+df['anticipation']+df['disgust']+df['fear']+df['joy']+df['negative']+df['positive']+df['sadness']+df['surprise']+df['trust']
# df = df[["word","sum"]]
# dic = df.set_index('word').to_dict()


In [16]:
df

,# POS,ID,PosScore,NegScore,SynsetTerms,Gloss
0,a,1740.0,0.125,0.000,able#1,(usually followed by `to') having the necessar...
1,a,2098.0,0.000,0.750,unable#1,(usually followed by `to') not having the nece...
2,a,2312.0,0.000,0.000,dorsal#2 abaxial#1,facing away from the axis of an organ or organ...
3,a,2527.0,0.000,0.000,ventral#2 adaxial#1,nearest to or facing toward the axis of an org...
4,a,2730.0,0.000,0.000,acroscopic#1,facing or on the side toward the apex
...,...,...,...,...,...,...
117655,v,2771888.0,0.000,0.125,fog_up#1,"get foggy; ""The windshield fogged up"""
117656,v,2771997.0,0.000,0.000,coal#1 char#1,"burn to charcoal; ""Without a drenching rain, t..."
117657,v,2772202.0,0.125,0.250,haze#1,"become hazy, dull, or cloudy"
117658,v,2772310.0,0.125,0.000,deflagrate#1,cause to burn rapidly and with great intensity...


In [17]:
for each in df["SynsetTerms",""][:50]:
    a = each.split()
    for each in a: 

# POS
ID
PosScore
NegScore
SynsetTerms
Gloss


In [ ]:
count = 0
for each in df["SynsetTerms"]:
    if len(each)>2 and each[-2]!= "#":
        count+=1


In [ ]:
# concat
# print(joy_train.shape,anger_train.shape)
# train = pd.concat([joy_train,anger_train])

In [ ]:
import nltk
nltk.download('sentiwordnet')
nltk.download('wordnet')
from nltk.corpus import sentiwordnet as swn


sentence = "Iphone6 camera is awesome for low light "
token = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(token)

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
token

['Iphone6', 'camera', 'is', 'awesome', 'for', 'low', 'light']

In [ ]:
tagged

[('Iphone6', 'NNP'),
 ('camera', 'NN'),
 ('is', 'VBZ'),
 ('awesome', 'JJ'),
 ('for', 'IN'),
 ('low', 'JJ'),
 ('light', 'NN')]

In [ ]:
breakdown = swn.senti_synset('Iphone6.NNP.03')


WordNetError: ignored

In [ ]:
breakdown.neg_score()

0.0

In [ ]:
breakdown.pos_score()

0.375

In [ ]:
# nltk.download('tagsets')
# nltk.help.upenn_tagset()


     |████████████████████████████████| 61kB 1.7MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53453 sha256=e57f97d94c6d5b4ccd936f48ae4a3645870f58e2e4126b02cb71eab6ae637de6
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


NameError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
